In [1]:
import numpy as np
import random
import copy
import time

In [2]:
b=np.zeros((8,8))
b[0:2,:]=1#player 1 
b[6:8,:]=-1#player 2
def matrix_to_nodes(mat,direction):
    node={}
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            n=[]
            if direction==1:
                if (i!=mat.shape[0]-1) and (j!=0):
                    ld_child=(i+1,j-1)
                    n.append(ld_child)
                if (i!=mat.shape[0]-1) and (j!=mat.shape[1]-1):
                    rd_child=(i+1,j+1)
                    n.append(rd_child)
                if (i!=mat.shape[0]-1):
                    d_child=(i+1,j)
                    n.append(d_child)
                node.update({(i,j):n})
            else:
                if (i!=0) and (j!=0):
                    ld_child=(i-1,j-1)
                    n.append(ld_child)
                if (i!=0) and (j!=mat.shape[1]-1):
                    rd_child=(i-1,j+1)
                    n.append(rd_child)
                if (i!=0):
                    u_child=(i-1,j)
                    n.append(u_child)
                node.update({(i,j):n})
                
    return node

class Board:
    B=np.zeros((8,8))
    B[0:2,:]=1#player 1 
    B[6:8,:]=-1#player 2
    mat=B
    def __init__(self, matr):#initialising the node with its coordinates in the matrix and value
        self.mat=matr   
    
    def get_current_state(self):
        return self.mat
    
    def update_state(self,mat):
        self.mat=mat
    
def is_gameover(m,player1,player2):
    p1=player1.get_pos_workers(m)
    p2=player2.get_pos_workers(m)
    if len(p1)==0 or len(p2)==0:
        return True
    for i in p1:
        if i[0]==7:
            return True
    for i in p2:
        if i[0]==0:
            return True
    return False 

def up_state(mat):
        g=Board(mat)
        return g
def next_state(game_state,move,P):
    m=game_state
    m[move[0][0]][move[0][1]]=0
    m[move[1][0]][move[1][1]]=P.getval()
    #Z=up_state(m)
    return m

        

class Player:
    player_val=1
    direction=1
    pos=[]
    count=0
    
    def __init__(self, val,direc):
        self.player_val=val
        self.direction=direc
    
    def getval(self):
        return self.player_val
    
    def get_pos_workers(self,m):
        #m=B.get_current_state()
        self.pos=zip(*np.where(m == self.player_val))
        return self.pos
    
    def child_nodes(self,m):
        n=matrix_to_nodes(m,self.direction)
        return n
        
    
    def player_avail_moves(self, m,opponent):#B :Board object
        
        player_pos=self.get_pos_workers(m)
        move=[]
        n=self.child_nodes(m)
        for node in player_pos:
            if len(n[node])!=0:
                for i in  n[node]:
                    if i!=(node[0]+1,node[1])and i!=(node[0]-1,node[1])and (m[i[0]][i[1]]!=self.player_val) :
                        move.append((node,i))
                    else:
                        if m[i[0]][i[1]]!=opponent and (m[i[0]][i[1]]!=self.player_val):
                            move.append((node,i))
        return move
    
    def players_reamining(self,m):
        p=self.get_pos_workers(m)
        return len(p)
    
    def expanded_nodes(self):
        self.count=self.count+1
    
    def Player_winner(self,m,player2):
        p1=self.get_pos_workers(m)
        p2=player2.get_pos_workers(m)
        if self.direction==1:
            for i in p1:
                if i[0]==7:
                    return True
        if self.direction==-1:
            for i in p2:
                if i[0]==0:
                    return True
        if len(p2)==0:
            return True
        return False 

    def check_rdiagplayer(self, m,node, opponent):
        x=False
        if self.direction==1:
            if (m[node[0]+1][node[1]+1]==opponent):
                x=True
        
        if self.direction==-1:
            if (m[node[0]-1][node[1]+1]==opponent):
                x=True
        return x
    
    def check_ldiagplayer(self,m,node, opponent):
        x=False
        if self.direction==1:
            if (m[node[0]+1][node[1]-1]==opponent):
                x=True
        if self.direction==-1:
            if (m[node[0]-1][node[1]-1]==opponent):
                x=True
        return x
    #def choose_move(self, ):

    def capture_opponent(self,B, mov, opponent):
        m=B.get_current_state()
        if m[mov[0]+1][mov[1]+1]==opponent:
            m[mov[0]+1][mov[1]+1]=self.player_val
        B.update_state(m) 
        return B.get_current_state()
    

In [3]:
# heuristics
#def move_ordering(moves,mat,player1,player 2):
  #  for i in moves
    #    if 
def O_H1(number_of_opponent_pieces_remaining) :
    val=2*(30 - number_of_opponent_pieces_remaining) + random.random()
    return val

def D_H1(number_of_own_pieces_remaining) :
    val=2*(number_of_own_pieces_remaining) + random.random()
    return val
def empty2_box_player(m,player):
    pos=player.get_pos_workers(m)
    c=0
    for i in pos:
        if player.direction==1:
            if i[0]<=5 and i[1]>=2 and i[1]<=5:
                if (m[i[0]+1][i[1]+1]==0) and (m[i[0]+2][i[1]+2]==0 ) and (m[i[0]+1][i[1]]==0)  and( m[i[0]+2][i[1]]==0 ) and (m[i[0]+1][i[1]-1]==0)  and (m[i[0]+2][i[1]-2]==0): 
                    c=c+1
                    return c
        if player.direction==-1:
            if i[0]>=2 and i[1]>=2 and i[1]<=5:
                if (m[i[0]-1][i[1]+1]==0) and (m[i[0]-2][i[1]+2]==0 ) and (m[i[0]-1][i[1]]==0)  and( m[i[0]-2][i[1]]==0 ) and (m[i[0]-1][i[1]-1]==0)  and (m[i[0]-2][i[1]-2]==0): 
                    c=c+1
                    return c
    return c

def empty1_box_player(m,player):
    pos=player.get_pos_workers(m)
    c=0
    for i in pos:
        if player.direction==1:
            if i[0]<=6 and i[1]>=1 and i[1]<=6:
                if (m[i[0]+1][i[1]+1]==0)  and (m[i[0]+1][i[1]]==0)   and (m[i[0]+1][i[1]-1]==0)  : 
                    c=c+1
                    return c
        if player.direction==-1:
            if i[0]>=1 and i[1]>=1 and i[1]<=6:
                if (m[i[0]-1][i[1]+1]==0)  and (m[i[0]-1][i[1]]==0)  and (m[i[0]-1][i[1]-1]==0) : 
                    c=c+1
                    return c
    return c

def empty_box_player(m,player,player2):
    pos=player.get_pos_workers(m)
    c=0
    for i in pos:
        if player.direction==1:
            if i[0]<=6 and i[1]>=1 and i[1]<=6:
                if (m[i[0]+1][i[1]+1]==player2.getval())  and (m[i[0]+1][i[1]]==player2.getval())   and (m[i[0]+1][i[1]-1]==player2.getval())  : 
                    c=c+1
                    return c
        if player.direction==-1:
            if i[0]>=1 and i[1]>=1 and i[1]<=6:
                if (m[i[0]-1][i[1]+1]==player2.getval())  and (m[i[0]-1][i[1]]==player2.getval())  and (m[i[0]-1][i[1]-1]==player2.getval()) : 
                    c=c+1
                    return c
    return c

In [4]:
#evaluation function
#offensive Heuristic 1 
def evaluate(game_state,player1,player2):
    x=player2.players_reamining(game_state)
    return O_H1(x)

#defensive Heuristic 1
def evaluate_D1(game_state,player1,player2):
    x=player1.players_reamining(game_state)
    return D_H1(x)

##offensive Heuristic 2
def evaluate_O2(game_state,player1,player2):
    clone1=copy.deepcopy(game_state)
    x=player2.players_reamining(clone1)
    y=3*(empty2_box_player(clone1,player1))+2*(empty1_box_player(clone1,player1))+2*O_H1(x)
    return y
    #for i in moves:
    #    if 
   # y=O_H1(x)#
def evaluate_D2(game_state,player1,player2):
    clone1=copy.deepcopy(game_state)
    x=player2.players_reamining(clone1)
    y=D_H1(x)+3*(empty2_box_player(clone1,player1))+2*(empty1_box_player(clone1,player1))+2*O_H1(x)-3*(empty_box_player(clone1,player1,player2))
    return y

In [5]:
#Alphabeta search
inf=9999999999
neg_inf=-9999999999
def Alphabeta(state,player1,player2,depth,evalfunc=None):
    
    clone1=copy.deepcopy(state)
    opponent=player2.getval()
    moves = player1.player_avail_moves(state,opponent)
    v= max_value(clone1,player1,player2,depth-1,neg_inf,inf,evalfunc)
    for move in moves:
        clone2= copy.deepcopy( clone1)
        clone = next_state(clone2,move,player1)
        player1.expanded_nodes()
        x=evalfunc(clone,player1,player2)
        if int(x)==int(v):
            return move
    return moves[0]

def max_value(state,player1,player2,depth,a,b,evalfunc=None):
    clone1=copy.deepcopy(state)
    if is_gameover(clone1,player1,player2)!=True or depth<=0:
        return evalfunc(clone1,player1,player2)
    else:
        v=neg_inf
        opponent=player2.getval()
        moves = player1.player_avail_moves(clone1,opponent)
        for move in moves:
            clone2= copy.deepcopy( clone1)
            clone = next_state(clone2,move,player1)
            clone4=copy.deepcopy( clone)
            player1.expanded_nodes()
            v=max(v, min_value(clone4,player1,player2,depth-1,a,b,evalfunc))
        if v>=b :
            return v
        a=max(a,v)
        return v

def min_value(state,player1,player2,depth,a,b,evalfunc):
    clone1=copy.deepcopy(state)
    if is_gameover(clone1,player1,player2)!=True or depth<=0:
        return evalfunc(clone1,player1,player2)
    else:
        v=inf
        opponent=player1.getval()
        moves = player2.player_avail_moves(clone1,opponent)
        for move in moves:
            clone2= copy.deepcopy( clone1)
            clone = next_state(clone2,move,player2)
            clone4=copy.deepcopy( clone)
            player1.expanded_nodes()
            v=min(v, max_value(clone4,player1,player2,depth-1,a,b,evalfunc))

        if v <=a :
            return v
        b=min(b, v)
        return v
    


In [6]:
inf=9999999999
neg_inf=-9999999999
def Alphabeta1(state,player1,player2,depth,evalfunc=None):
    
    clone1=copy.deepcopy(state)
    opponent=player2.getval()
    moves = player1.player_avail_moves(state,opponent)
    v= max1_value(clone1,player1,player2,depth-1,neg_inf,inf,evalfunc)
    for move in moves:
        clone2= copy.deepcopy( clone1)
        clone = next_state(clone2,move,player1)
        player1.expanded_nodes()
        x=evalfunc(clone,player1,player2)
        if int(x)==int(v):
            return move
    return moves[0]

def max1_value(state,player1,player2,depth,a,b,evalfunc=None):
    clone1=copy.deepcopy(state)
    if is_gameover(clone1,player1,player2)==True or depth<=0:
        return evalfunc(clone1,player1,player2)
    else:
        v=neg_inf
        opponent=player2.getval()
        moves = player1.player_avail_moves(clone1,opponent)
        for move in moves:
            clone2= copy.deepcopy( clone1)
            clone = next_state(clone2,move,player1)
            clone4=copy.deepcopy( clone)
            player1.expanded_nodes()
            v=max(v, min1_value(clone4,player1,player2,depth-1,a,b,evalfunc))
        if v>=b :
            return v
        a=max(a,v)
        return v

def min1_value(state,player1,player2,depth,a,b,evalfunc):
    clone1=copy.deepcopy(state)
    if is_gameover(clone1,player1,player2)==True or depth<=0:
        return evalfunc(clone1,player1,player2)
    else:
        v=inf
        opponent=player1.getval()
        moves = player2.player_avail_moves(clone1,opponent)
        for move in moves:
            clone2= copy.deepcopy( clone1)
            clone = next_state(clone2,move,player2)
            clone4=copy.deepcopy( clone)
            player1.expanded_nodes()
            v=min(v, max1_value(clone4,player1,player2,depth-1,a,b,evalfunc))

        if v <=a :
            return v
        b=min(b, v)
        return v
    
    

In [7]:
#yaaaaas finally the game starts
x=True
count=0
p1=Player(1,1)
p2=Player(-1,-1)
S=Board(b)
m2=S.get_current_state()
print S.get_current_state()
start_time = time.clock()
while (x):
    m2=S.get_current_state()
    x1=Alphabeta1(m2,p1,p2,4,evaluate_O2) 
    print x1
    next_state(m2,x1,p1)
    count+=1
    print S.get_current_state()
    if is_gameover(m2,p1,p2):
        x=False
        break
    m2=S.get_current_state()
    m3=copy.deepcopy(m2)
    #while (x):     
    x2=Alphabeta(m3,p2,p1,4,evaluate_D2)
    print x2
    next_state(m3,x2,p2)
    count+=1
    m4=copy.deepcopy(m3)
    S.update_state(m4)
    print S.get_current_state()
    if is_gameover(m4,p1,p2):
        x=False
        break
endtime=time.clock()- start_time
final_board_state=S.get_current_state()
#GameOver
print S.get_current_state() 
    #count=count+1
print count

if p1.Player_winner(final_board_state,p2) :
    winner=str('player1')
else:
    winner=str('player2')

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [-1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1.]]
((1, 0), (2, 1))
[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 0.  1.  1.  1.  1.  1.  1.  1.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [-1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1.]]
((6, 1), (5, 1))
[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 0.  1.  1.  1.  1.  1.  1.  1.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.  0.  0.]
 [-1.  0. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1.]]
((0, 0), (1, 0))
[[ 0.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 

In [8]:
print 'player1:''1'' and player2:''-1'''
print 'Final state of the board:','\n',S.get_current_state(),'\n' 
print 'Winner:',winner
print 'The number of opponent workers captured by  player1:',(16-p2.players_reamining(final_board_state))
print 'The number of opponent workers captured by  player2:',(16-p1.players_reamining(final_board_state))
print 'Total number of moves required till the win:', count
print 'Total number of game tree nodes expanded by  player1:',p1.count
print 'Total number of game tree nodes expanded by  player2:',p2.count
print 'Average number of nodes expanded per move:',int((p1.count+p2.count)/count)
print 'Average amount of time to make a move:',endtime/(count)+0.,'seconds'

#####Yassssssssssssssss it works

player1:1 and player2:-1
Final state of the board: 
[[ 0.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  1.  1.]
 [ 1.  1.  1.  1.  0.  1.  0.  1.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0. -1.]
 [ 0. -1.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0. -1.  0.  0. -1. -1.]] 

Winner: player2
The number of opponent workers captured by  player1: 9
The number of opponent workers captured by  player2: 4
Total number of moves required till the win: 78
Total number of game tree nodes expanded by  player1: 491898
Total number of game tree nodes expanded by  player2: 435252
Average number of nodes expanded per move: 11886
Average amount of time to make a move: 1.28354884615 seconds


In [9]:
x1=Player(1,1)
x2=Player(-1,-1)
B=Board(b)
 
m=B.get_current_state()
print B.get_current_state() 
opponent=x1.getval()
y=x2.player_avail_moves(m,opponent)
print y

m1=copy.deepcopy(m)
G=next_state(m1,((1,1),(2,1)),x2)
print B.get_current_state()  
print is_gameover(m,x1,x2)

x=Alphabeta(m,x2,x1,3,evaluate)
print x2.count
next_state(m,x,x2)
print B.get_current_state() 
print x

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 0.  1.  1.  1.  1.  1.  1.  1.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [-1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1.]]
[((6, 0), (5, 1)), ((6, 0), (5, 0)), ((6, 1), (5, 0)), ((6, 1), (5, 2)), ((6, 1), (5, 1)), ((6, 2), (5, 1)), ((6, 2), (5, 3)), ((6, 2), (5, 2)), ((6, 3), (5, 2)), ((6, 3), (5, 4)), ((6, 3), (5, 3)), ((6, 4), (5, 3)), ((6, 4), (5, 5)), ((6, 4), (5, 4)), ((6, 5), (5, 4)), ((6, 5), (5, 6)), ((6, 5), (5, 5)), ((6, 6), (5, 5)), ((6, 6), (5, 7)), ((6, 6), (5, 6)), ((6, 7), (5, 6)), ((6, 7), (5, 7))]
[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 0.  1.  1.  1.  1.  1.  1.  1.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [-1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1.]]
False
529
[[ 1.  1.  1.  1.  1.  1.  1.  

In [10]:
def hi(y,x):
        return x,y
    

In [11]:
a,b=hi(3,1)
x=b
print x

3


In [12]:
x=5.898
int(x)

5

In [13]:
def(x,y)
    

SyntaxError: invalid syntax (<ipython-input-13-1f5003bf3f6f>, line 1)